In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [2]:
CHROMEDRIVER_PATH = r"C:\ChromeDriver\chromedriver-win64\chromedriver.exe"

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
url = 'https://vietnamairport.vn/thong-tin-lich-bay'
driver.get(url)

time.sleep(5)

try:
    wait = WebDriverWait(driver, 10)
    search_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "btn-filter")))
    search_button.click()

    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#flight_table tbody tr")))

    column_headers = [th.text.strip() for th in driver.find_elements(By.CSS_SELECTOR, "table.table-striped thead th")]
    rows = driver.find_elements(By.CSS_SELECTOR, "table.table-striped tbody tr")
    table_data = []
    
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        table_data.append([cell.text.strip() for cell in cells])

    driver.quit()
    df = pd.DataFrame(table_data, columns=column_headers)

    df.to_csv("flights.csv", index=False)

    print("Crawl successfully!")

except Exception as e:
    print("Failed:", e)
    driver.quit()

✅ Crawl dữ liệu thành công! Dữ liệu đã được lưu vào 'flight_data.xlsx'.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime, timedelta
import time, os

driver = webdriver.Chrome()
start_date = datetime(2025, 4, 8)
end_date = datetime(2025, 4, 10)

url = 'https://vietnamairport.vn/thong-tin-lich-bay'

for i in range((end_date - start_date).days + 1):
    current_date = start_date + timedelta(days=i)
    print(f"Crawling for date: {current_date.strftime('%Y-%m-%d')}")

    driver.get(url)
    time.sleep(3)

    try:
        date_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "flight_date")))
        driver.execute_script("arguments[0].value = arguments[1]", date_input, current_date.strftime("%Y-%m-%d"))

        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "btn-filter")))
        search_button.click()

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#flight_table tbody tr")))

        column_headers = [th.text.strip() for th in driver.find_elements(By.CSS_SELECTOR, "table.table-striped thead th")]
        rows = driver.find_elements(By.CSS_SELECTOR, "table.table-striped tbody tr")
        table_data = []

        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            data_row = [cell.text.strip() for cell in cells]
            data_row.append(current_date.strftime('%Y-%m-%d'))
            table_data.append(data_row)

        column_headers.append("Flight Date")
        df = pd.DataFrame(table_data, columns=column_headers)

        file_path = "flights.csv"
        file_exists = os.path.isfile(file_path)
        df.to_csv(file_path, mode='a', index=False, header=not file_exists)

        print(f"Done: {current_date.strftime('%Y-%m-%d')}")

    except Exception as e:
        print(f"Failed: {e}")
        continue

driver.quit()
print("Crawl completed!")

Crawling for date: 2025-04-08
Done: 2025-04-08
Crawling for date: 2025-04-09
Done: 2025-04-09
Crawling for date: 2025-04-10
Done: 2025-04-10
Crawl completed!
